In [4]:
import json
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
import os

# 출력 디렉토리 설정
output_dir = r"C:\baramproject\tidal_visualize"
os.makedirs(output_dir, exist_ok=True)  # 디렉토리가 없으면 생성

# JSON 파일 경로 기본 패턴
base_path = r"C:\baramproject\tidal_database\tidal_20180101_{:04d}.json"

# 하루 동안 30분 간격으로 파일 생성 (00:00 ~ 23:30)
hours = [f"{h:02d}{m:02d}" for h in range(24) for m in [0, 30]]

for time in hours:
    # JSON 파일 경로 생성
    json_file_path = base_path.format(int(time))

    # JSON 파일이 존재하는지 확인
    if not os.path.exists(json_file_path):
        print(f"파일이 존재하지 않습니다: {json_file_path}")
        continue

    # JSON 파일 읽기
    with open(json_file_path, 'r') as f:
        data = json.load(f)

    # 데이터 추출
    lons = [float(item['pre_lon']) for item in data['result']['data']]
    lats = [float(item['pre_lat']) for item in data['result']['data']]
    directions = [float(item['current_dir']) for item in data['result']['data']]
    speeds = [float(item['current_speed']) for item in data['result']['data']]

    # 유향을 x, y 성분으로 변환 (북쪽이 0도인 방위각 기준)
    u = [np.sin(np.deg2rad(dir)) for dir in directions]
    v = [np.cos(np.deg2rad(dir)) for dir in directions]

    # 지도 설정
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
    ax.set_extent([120, 127, 30, 38], crs=ccrs.PlateCarree())

    # 지도 특징 추가
    ax.add_feature(cfeature.LAND)
    ax.add_feature(cfeature.OCEAN)
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.BORDERS, linestyle=':')

    # 화살표 그리기 (유속에 따라 색상만 변경, 크기는 일정)
    norm = plt.Normalize(min(speeds), max(speeds))
    cmap = plt.get_cmap('coolwarm')

    # 화살표 크기 고정 및 색상으로 유속 표시
    quiver = ax.quiver(lons, lats, u, v, speeds, cmap=cmap, norm=norm, scale=20, width=0.005, headwidth=3, headlength=5)

    # 색상 바 추가
    cbar = plt.colorbar(quiver, ax=ax, orientation='vertical', pad=0.05, aspect=50)
    cbar.set_label('Current Speed (cm/s)')

    # 제목 추가 (시간 포함)
    plt.title(f'Tidal Current Visualization - 2018-01-01 {time[:2]}:{time[2:]}')

    # 이미지 저장
    output_file = os.path.join(output_dir, f'tidal_vis_20180101_{time}.png')
    plt.savefig(output_file, dpi=300, bbox_inches='tight')
    plt.close(fig)  # 메모리 관리를 위해 figure 닫기

    print(f"저장 완료: {output_file}")

print("모든 시각화 이미지 저장 완료!")

저장 완료: C:\baramproject\tidal_visualize\tidal_vis_20180101_0000.png
저장 완료: C:\baramproject\tidal_visualize\tidal_vis_20180101_0030.png
저장 완료: C:\baramproject\tidal_visualize\tidal_vis_20180101_0100.png
저장 완료: C:\baramproject\tidal_visualize\tidal_vis_20180101_0130.png
저장 완료: C:\baramproject\tidal_visualize\tidal_vis_20180101_0200.png
저장 완료: C:\baramproject\tidal_visualize\tidal_vis_20180101_0230.png
저장 완료: C:\baramproject\tidal_visualize\tidal_vis_20180101_0300.png
저장 완료: C:\baramproject\tidal_visualize\tidal_vis_20180101_0330.png
저장 완료: C:\baramproject\tidal_visualize\tidal_vis_20180101_0400.png
저장 완료: C:\baramproject\tidal_visualize\tidal_vis_20180101_0430.png
저장 완료: C:\baramproject\tidal_visualize\tidal_vis_20180101_0500.png
저장 완료: C:\baramproject\tidal_visualize\tidal_vis_20180101_0530.png
저장 완료: C:\baramproject\tidal_visualize\tidal_vis_20180101_0600.png
저장 완료: C:\baramproject\tidal_visualize\tidal_vis_20180101_0630.png
저장 완료: C:\baramproject\tidal_visualize\tidal_vis_20180101_0700

In [5]:
import json
import numpy as np

# JSON 파일 경로
json_file_path = r"C:\baramproject\tidal_database\tidal_20180101_0000.json"

# JSON 파일 읽기
with open(json_file_path, 'r') as f:
    data = json.load(f)

# 위도와 경도 추출
lats = sorted([float(item['pre_lat']) for item in data['result']['data']])
lons = sorted([float(item['pre_lon']) for item in data['result']['data']])

# 인접한 값 간 차이 계산
lat_diffs = np.diff(lats)
lon_diffs = np.diff(lons)

# 최소 간격과 평균 간격 출력
min_lat_diff = np.min(lat_diffs) if len(lat_diffs) > 0 else 0
min_lon_diff = np.min(lon_diffs) if len(lon_diffs) > 0 else 0
avg_lat_diff = np.mean(lat_diffs) if len(lat_diffs) > 0 else 0
avg_lon_diff = np.mean(lon_diffs) if len(lon_diffs) > 0 else 0

print(f"위도 최소 간격: {min_lat_diff:.5f}도")
print(f"경도 최소 간격: {min_lon_diff:.5f}도")
print(f"위도 평균 간격: {avg_lat_diff:.5f}도")
print(f"경도 평균 간격: {avg_lon_diff:.5f}도")

# 데이터가 규칙적인지 확인 (최소 간격과 평균 간격이 비슷한지)
if np.allclose(lat_diffs, min_lat_diff, rtol=0.1):
    print("위도 데이터는 규칙적인 간격을 가지는 것으로 보입니다.")
else:
    print("위도 데이터는 불규칙적인 간격을 가집니다.")
if np.allclose(lon_diffs, min_lon_diff, rtol=0.1):
    print("경도 데이터는 규칙적인 간격을 가지는 것으로 보입니다.")
else:
    print("경도 데이터는 불규칙적인 간격을 가집니다.")

위도 최소 간격: 0.00001도
경도 최소 간격: 0.00001도
위도 평균 간격: 0.00441도
경도 평균 간격: 0.00395도
위도 데이터는 불규칙적인 간격을 가집니다.
경도 데이터는 불규칙적인 간격을 가집니다.
